# MASK-Maintaining Data Privacy in Association Rule Mining

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
link)

In [1]:

test_ds_path = ".\..\Datasets\store_data.csv"

with open(test_ds_path, "r") as f:

    lines = f.readlines()
    inventory = list()
    tuples = list()

    for line in lines:
        transaction = line.strip().split(",")
        #print(transaction)
        tuples.append(transaction)
        for element in transaction:
            if element.strip().replace(' ','_').replace('&','and') not in inventory:
                inventory.append(element.strip().replace(' ','_').replace('&','and'))

In [ ]:
print(len(inventory))

In [ ]:

print(inventory)
inventory.pop()
inventory.sort()
print(inventory)

In [ ]:
print(len(inventory))

In [5]:
import pandas as pd


In [6]:
test_dataset = pd.DataFrame(
    [[1 if item in row else 0 for item in inventory] for row in tuples],
    columns=inventory
)

In [ ]:
test_dataset.head(3)

In [ ]:
test_dataset.columns

In [ ]:
type(test_dataset.columns)

In [ ]:
test_dataset['zucchini'].sum()


In [ ]:
for tuple in test_dataset.itertuples():
    print(tuple)
    print(int( getattr(tuple, 'almonds')))
    print(type(tuple))
    break

In [ ]:
import numpy as np
for tuple in test_dataset.itertuples(False):
    print(tuple)
    test_tuple = tuple
    print(type(tuple))
    print(np.asarray(tuple))
    break

dataset_mini = test_dataset.head(3)


print(dataset_mini)

In [ ]:
dataset_mini.loc[0,'almonds']

In [14]:

from pandas import DataFrame


def support(T: DataFrame ,X_U_Y: list | dict):
    '''
    Parameters:
    T (dataframe)
    X_U_Y (list | dict) : name of the attributes considered (X U Y)
    Return:
    float: support of the attributes in the dataset
    '''
    for attribute in X_U_Y:
        if attribute not in T.columns:
             return ValueError
    count = 0
    for tuple in T.itertuples(False):
        contained = True
        for attribute in X_U_Y:
            try: 
                if getattr(tuple,attribute) == 0:
                    contained = False
                    break
            except AttributeError:
                contained = False,
                break

        if contained:
            count += 1 
    return count/len(T)
    

In [ ]:
support(test_dataset,['herb_and_pepper'])

In [16]:

def confidence(T: DataFrame, X: list | dict, Y: list | dict):
    '''
    Parameters:
    T (dataframe)
    X ( list | dict): X part of X ==> Y rule
    X (list | disct): Y part of X ==> Y rule
    '''
    if len(X)+len(Y) > len(T.columns):
        return ValueError
    for item in X:
        if Y.__contains__(item):
            return ValueError # XY = empty set
    
    countX=0
    countY=0
    for tuple in T.itertuples():
        containedX = True
        for attribute in X:
            if getattr(tuple,attribute) == 0:
                containedX = False
                break
        if containedX:
            countX += 1
            containedY=True
            for attribute in Y:
                if attribute == '':
                    containedY = False
                    break
                if getattr(tuple,attribute) == 0:
                    containedY = False
                    break
            if containedY:
                countY += 1
    return countY/countX    

In [ ]:
confidence(test_dataset,['spaghetti','champagne'],['cookies'])
#spaghetti,champagne,cookies

In [41]:
def Apriori(items, dataset, min_sup, levels: int = None):
    if levels is None:
        levels = len(items)
    '''
    rules[0] = empty set
    rules[1] = rules of length 1
    rules[2] = rules of length 2


    rules[len(items)] = items
    ...
    '''
    rules = [
        [],
        [ [item] for item in items ]
    ]


    # iterate over all the possible rules length from 1 to len(items)
    for i in range(1, levels+1):
        print(f"Apriori Level: {i}")
        # remove all the rules in rules[i]
        # that don't have a support of at least min_sup

        #print(f"RULES[{i}] BEFORE", rules[i])

        rules[i] = [
            rule
            for rule in rules[i]
            if support(dataset, rule) >= min_sup
        ]
        
        '''
        print(f"RULES[{i}] SUPPORT", [
            support(dataset, rule)
            for rule in rules[i]
        ]
        )'''


        #print(f"RULES[{i}] AFTER", rules[i])

        if len(rules[i]) == 0:
            break


        # generate all the possibile 
        # rules with i+1 elements
        rules.append([]) # create the element rules[i+1]

        #print("RULE[i]", rules[i])
        #print("RULE[i+1]", rules[i+1])
        for rule in rules[i]:
            for item in items:

                # skip if item is already inside the rule
                if item in rule:
                    continue

                rules[i+1].append ( rule + [item] )
                # create a new rule composed of rule + [item]


        #print("RULE[i+1]", rules[i+1])
    
        #print()
        #print()
        #print()

    return rules

In [19]:

#rules = Apriori(inventory,test_dataset,0.004, 4)

In [20]:
#print(rules)

In [ ]:
import random
random.random()

In [ ]:
support_vector = [[item,support(test_dataset,[item])] for item in inventory]
print(support_vector)
mean_support = np.mean([sublist[1] for sublist in support_vector])
print(mean_support)

frequent_item = []
infrequent_item = []


for item in support_vector:
    if item[1]>mean_support:
        frequent_item.append(item)
    else:
        infrequent_item.append(item)



support_vector
print(support_vector)
print(frequent_item)
print(infrequent_item)

print(len(support_vector))
print(len(frequent_item))
print(len(infrequent_item))

In [23]:
import random

def MASK_Distortion(dataset: DataFrame, p: float):
    '''
    MASK
    
    choose a probability p

    2 event
    P(x = true) -> 1-p % -> we add or remove an element in the transaction
    P(x = false) -> p% -> the transaction remains the same
    '''
    distorted_dataset = dataset.copy(deep=True)
    for i in range(0,len(distorted_dataset)):
        for column in distorted_dataset.columns:
            event = random.random()
            if event > p:
                distorted_dataset.loc[i, column] = int(not dataset.loc[i, column])

    return distorted_dataset

        



In [24]:
def MASK_frequent_infrequent_items(support_vector,sup_min):
    frequent_item = []
    infrequent_item = []

    for item in support_vector:
        if item[1]>sup_min:
            frequent_item.append(item[0])
        else:
            infrequent_item.append(item[0])
    return frequent_item,infrequent_item

In [25]:
p = 0.9
distorted_test_dataset = MASK_Distortion(test_dataset,p)

In [26]:
def hammingDistanceBitwise(a:int,b:int):
    return (a^b).bit_count()

In [ ]:
hammingDistanceBitwise(int(1),int(2))

In [ ]:
import numpy as np
import math
M = np.diag([math.pow(2,3) for i in range(0,3)])
print(M)

In [29]:
import numpy as np
import math
def computeM(size: int, p: float):
    max_exp = int(math.log2(size))
    M = np.diag([math.pow(p,max_exp) for i in range(size)])
    for i in range(0,size):
        for j in range(i+1,size):
            difference = hammingDistanceBitwise(i,j)
            M[i][j] = math.pow(1-p,difference)*math.pow(p,max_exp-difference)

    for i in range(1,size):
        for j in range(i,-1,-1):
            M[i][j] = M[j][i]

    return M


In [ ]:
M_test = computeM(int(math.pow(2,2)),0.1)
print(M_test)
print(sum(M_test[0]))

In [31]:
def computeLinC_D(dataset: DataFrame,rule: list | dict):
    linC_D = np.zeros(len(rule)+1,dtype=int)
    reducted_ds = dataset[rule] # posso prendere solo le colonne interessanti
    for tuple in reducted_ds.itertuples(False):
        '''sum = 0
        for attribute in rule:
            sum += getattr(tuple,attribute) # o farlo qua con il for
            linC_D[sum] += 1
        '''
        
        linC_D[np.sum(tuple,axis=0)] += 1
    
    return linC_D
    

In [32]:
def vectormatrixProdMod(linC_D,matrix):
    size = matrix.shape[0]
    row = matrix[size-1]
    sum=0
    for j in range(len(linC_D)):
        index = int(math.pow(2,j))-1
        #print(str(row[index])+" * "+str(linC_D[j]))
        sum += row[index]*linC_D[j]
    return sum

In [ ]:
from numpy.linalg import inv

M_test = inv(computeM(2**2,p))
print(M_test)
print(sum(M_test[3]))

In [34]:
from numpy.linalg import inv

def MASK_Support(dataset:DataFrame, rule, p:float, M_inv = None):
    if M_inv is None:
        M_inv = inv(
            computeM(
                size=int(math.pow(2,len(rule))),
                p=p
            )
        )

    linC_D = computeLinC_D(dataset,rule)

    #print(linC_D)
    #print(M_inv)

    C_T_11 = vectormatrixProdMod(linC_D,M_inv)

    #print(C_T_11)

    return C_T_11/len(dataset)
    

In [74]:

import itertools



def MASK_Apriori(dataset: DataFrame, p: float, min_sup,levels: int = None):
    if levels is None:
        levels = len(dataset.columns)
    rules = [
        [],
        [ [item] for item in dataset.columns ]
    ]

    for i in range(2, levels+1):
        print(f"Constructing Level: {i}")
        combinations = list(itertools.combinations(dataset.columns,i))
        rule_i = [list(c) for c in combinations]
        rules.append(rule_i)
    

                    # create a new rule composed of rule + [item]
        

    for i in range(1,levels+1):
        print(f"Analysing Level: {i}")
        size = int(math.pow(2,i))
        M_inv = inv(
            computeM(size,p)
        )
        
        rules[i] = [
            rule
            for rule in rules[i]
            if MASK_Support(dataset,rule,p,M_inv) >= min_sup
        ]

    return rules




    

In [ ]:
reduced_inventory = inventory[0:5]
print(reduced_inventory)

In [ ]:
reduced_test_ds = test_dataset.iloc[0:1000, 0:5]
print(reduced_test_ds)

In [ ]:
support_vector_reduced = [[item,support(reduced_test_ds,[item])] for item in reduced_inventory]
print(support_vector_reduced)
mean_reduced_support = np.mean([sublist[1] for sublist in support_vector_reduced])
print(mean_reduced_support)

In [39]:
p = 0.9
reduced_distorted_test_ds = MASK_Distortion(reduced_test_ds,p)

In [ ]:
min_sup = 0.001
rules = Apriori(reduced_inventory,reduced_test_ds,min_sup)
print(rules)

In [ ]:
MASK_rules = MASK_Apriori(reduced_distorted_test_ds,p,min_sup)
print(MASK_rules)